In [1]:
# Python natives
import os

os.chdir("/home/tim/Development/OCPPM/")
import lightgbm as lgb
import pprint
import numpy as np
import pandas as pd
import re
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from utilities import evaluation_utils

In [2]:
event_attributes = ["ea2", "ea4", "resource_ce", "resource_multi"]
eft_in_file = "data/CS/feature_encodings/baselines/EFT/eft.csv"

In [3]:
eft_dtypes = {f"event_{ea}": int for ea in event_attributes}
events_df = pd.read_csv(eft_in_file, sep=";")  # , dtype=eft_dtypes)
events_df.head()

,event_preceding_activitiesFinishTask,event_preceding_activitiesUpdateObjectState,event_preceding_activitiesReallocateTask,event_preceding_activitiesSetGlobalTarget,event_preceding_activitiesCreateObject,event_preceding_activitiesSetTarget,event_preceding_activitiesPerformAction,event_preceding_activitiesAttachDocument,event_preceding_activitiesLinkObjects,event_preceding_activitiesSetTargetDue,...,event_preceding_activitiesSetGlobalTargetDue,event_elapsed_time,event_synchronization_time,event_previous_type_countkrs,event_previous_type_countkrv,event_previous_type_countcv,event_ea2,event_ea4,event_resource_ce,event_resource_multi
0,1.739661,-0.37484,-0.388706,-0.264215,-0.235843,-0.184832,-0.188517,-0.184767,-0.221486,-0.163067,...,-0.127551,0.154136,-0.010803,0.600036,0.814806,-0.203715,-0.770879,6.0,-0.625678,-0.413848
1,-0.574810,-0.37484,-0.388706,-0.264215,-0.235843,-0.184832,5.304553,-0.184767,-0.221486,-0.163067,...,-0.127551,0.154136,-0.010803,0.600036,0.814806,-0.203715,-0.770879,6.0,-0.627289,2.416347
2,-0.574810,-0.37484,-0.388706,-0.264215,-0.235843,-0.184832,-0.188517,5.412227,-0.221486,-0.163067,...,-0.127551,1.912127,-0.010803,0.600036,0.814806,-0.203715,-0.431713,4.0,-0.624209,-0.413848
3,1.739661,-0.37484,-0.388706,-0.264215,-0.235843,-0.184832,-0.188517,-0.184767,-0.221486,-0.163067,...,-0.127551,1.912127,-0.010803,0.600036,0.814806,-0.203715,-0.431713,4.0,-0.627342,2.416347
4,-0.574810,-0.37484,2.572599,-0.264215,-0.235843,-0.184832,-0.188517,-0.184767,-0.221486,-0.163067,...,-0.127551,1.911374,-0.010803,0.600036,0.814806,-0.203715,-0.431713,4.0,-0.624209,-0.413848


In [4]:
# events_df = events_df[events_df["event_ea4"] != 0]
# events_df["event_ea4"] = events_df["event_ea4"] - 1

In [5]:
# make train test split
X, y = (
    events_df.drop(columns=["event_ea4"]),
    events_df.loc[:, "event_ea4"],
)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.3, random_state=42
)

train_data = lgb.Dataset(
    X_train,
    label=y_train,
)
valid_data = lgb.Dataset(
    X_valid,
    label=y_valid,
)

In [ ]:
events_df["event_ea4"].value_counts(normalize=True).sort_index()

In [7]:
params = {
    "num_threads": 4,
    "objective": "multiclass",
    "num_class": 7,
    "metric": ["multi_logloss"],
    "num_boost_round": 100,
    "stopping_rounds": 100,
}

bst = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    callbacks=[lgb.early_stopping(params["stopping_rounds"])],
)

/home/tim/Development/OCPPM/.env/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.088643 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 587
[LightGBM] [Info] Number of data points in the train set: 1747734, number of used features: 21
[LightGBM] [Warning] Unknown parameter: stopping_rounds
[LightGBM] [Info] Start training from score -0.696330
[LightGBM] [Info] Start training from score -3.402001
[LightGBM] [Info] Start training from score -3.858840
[LightGBM] [Info] Start training from score -3.586328
[LightGBM] [Info] Start training from score -2.157827
[LightGBM] [Info] Start training from score -2.013527
[LightGBM] [Info] Start training from score -1.769623
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best i

In [8]:
y_train_probs = bst.predict(X_train)
y_valid_probs = bst.predict(X_valid)

y_train_preds = np.apply_along_axis(
    evaluation_utils.get_preds_from_probs, axis=1, arr=y_train_probs
)
y_valid_preds = np.apply_along_axis(
    evaluation_utils.get_preds_from_probs, axis=1, arr=y_valid_probs
)

In [10]:
# Run model evaluation, TODO: store this somewhere (in JSON)
eval_train = evaluation_utils.get_evaluation(
    y_train, y_train_preds, classification=True
)
eval_valid = evaluation_utils.get_evaluation(
    y_valid, y_valid_preds, classification=True
)
experiment_settings = {"experiment_settings": params}
evaluation_report = {"train": eval_train, "validation": eval_valid}
pprint.pprint(evaluation_report)

{'train': {'report': {'0.0': {'f1-score': 0.7668347470532847,
                              'precision': 0.6768382785746039,
                              'recall': 0.8844344442020916,
                              'support': 871090},
                      '1.0': {'f1-score': 0.27398160315374503,
                              'precision': 0.5179381837290403,
                              'recall': 0.18625345725034786,
                              'support': 58211},
                      '2.0': {'f1-score': 0.134317556809958,
                              'precision': 0.6065359477124183,
                              'recall': 0.07552083333333333,
                              'support': 36864},
                      '3.0': {'f1-score': 0.04105512887530661,
                              'precision': 0.7699849170437406,
                              'recall': 0.021089812443195903,
                              'support': 48412},
                      '4.0': {'f1-score': 0.31296236353470